In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs, get_unit_tbl
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed
import pickle

In [2]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id'].tolist()
session_list[-13:]

['behavior_785956_2025-05-20_14-15-19',
 'behavior_785956_2025-05-21_13-42-02',
 'behavior_785956_2025-05-23_13-45-21',
 'behavior_784806_2025-06-17_14-59-23',
 'behavior_784806_2025-06-18_13-39-50',
 'behavior_784806_2025-06-20_13-39-16',
 'behavior_791691_2025-06-24_13-21-29',
 'behavior_791691_2025-06-25_14-06-10',
 'behavior_791691_2025-06-26_13-39-26',
 'behavior_791691_2025-06-27_13-54-30',
 'behavior_784803_2025-07-01_13-58-26',
 'behavior_784803_2025-07-02_13-41-41',
 'behavior_784803_2025-07-03']

In [3]:
# combine opto pdfs
data_type = 'curated'
def process(session, data_type):
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
        print(session)
        combine_pdf_big(pdf_dir, output_pdf)
Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list[-14:-3])
# for session in session_list[-14:-3]:
#     try:
#         process(session, data_type)
#         plt.close('all')
#     except:
#         print(f'Failed to process {session}')

KeyboardInterrupt: 

In [4]:
# from joblib import Parallel, delayed
# session_list = ['behavior_751181_2025-02-25_12-12-35',
#                 'behavior_751181_2025-02-26_11-51-19']
data_type = 'curated'
def process(session, data_type='raw'):
    plt.close('all')
    session_dir = session_dirs(session)
    print(session_dir[f'curated_dir_{data_type}'])
    if session_dir[f'curated_dir_{data_type}'] is None:
        return None
    print(f'{session} start')
    # beh_and_time_alignment(session) # plot behavior, check alignment between behavior, sound, and ephys
    # plt.close('all')
    ephys_opto_preprocessing(session, data_type, 'soma') 
    # # redo spiketimes if not aligned, make opto_table (laser time in nidaq); 
    # # generate a pkl file with opto_resp/lat to each condition
    # plt.close('all')
    ephys_opto_crosscorr(session, data_type)
    # # separate session into opto/beh, cal cross-correlation
    # plt.close('all')
    opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    # recal wf based on conditions. (pre/post, power), cal similarity, euclidean distance, correlation with spont
    # pay attention to difference early and late in the session
    
    # opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.3, lat_thresh=0.02, save=True)    
    # # save as pickle
    # with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
    #     opto_tagging_df.to_pickle(f)
    
    # if '751004' in session:
    #     plot_session_opto_drift(session, data_type, update_csv=False)
    # else:     
    #     plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        
 
# Parallel(n_jobs=6)(delayed(process)(session, data_type = data_type) for session in session_list[-13:])

for session in session_list[-13:]:
    print(session)
    session_dir = session_dirs(session)
    # if session_dir['curated_dir_curated'] is not None:
    process(session, data_type=data_type) 
    # elif  session_dir['curated_dir_raw'] is not None:
    #     print('Raw')
    #     process(session, data_type='raw')
# process(session_list[-13])
        
 

behavior_782394_2025-04-22_10-53-28
/root/capsule/data/behavior_782394_2025-04-22_10-53-28_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_782394_2025-04-22_10-53-28 start
behavior_782394_2025-04-22_10-53-28 stop
behavior_782394_2025-04-23_10-51-17
/root/capsule/data/behavior_782394_2025-04-23_10-51-17_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_782394_2025-04-23_10-51-17 start


compute_waveforms (workers: 5 processes):   0%|          | 0/3950 [00:00<?, ?it/s]

behavior_782394_2025-04-23_10-51-17 stop
behavior_782394_2025-04-24_12-07-34
/root/capsule/data/behavior_782394_2025-04-24_12-07-34_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_782394_2025-04-24_12-07-34 start


compute_waveforms (workers: 5 processes):   0%|          | 0/4543 [00:00<?, ?it/s]

behavior_782394_2025-04-24_12-07-34 stop
behavior_782394_2025-04-25_11-13-21
/root/capsule/data/behavior_782394_2025-04-25_11-13-21_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_782394_2025-04-25_11-13-21 start


...

behavior_785956_2025-05-20_14-15-19 stop
behavior_785956_2025-05-21_13-42-02
/root/capsule/data/behavior_785956_2025-05-21_13-42-02_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_785956_2025-05-21_13-42-02 start


compute_waveforms (workers: 5 processes):   0%|          | 0/4509 [00:00<?, ?it/s]

behavior_785956_2025-05-21_13-42-02 stop
behavior_785956_2025-05-23_13-45-21
/root/capsule/data/behavior_785956_2025-05-23_13-45-21_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1
behavior_785956_2025-05-23_13-45-21 start


compute_waveforms (workers: 5 processes):   0%|          | 0/4991 [00:00<?, ?it/s]

behavior_785956_2025-05-23_13-45-21 stop


In [3]:
session = 'behavior_782394_2025-04-22_10-53-28'
session_dir = session_dirs(session)

In [4]:
session_dir['curated_dir_raw']

'/root/capsule/data/behavior_782394_2025-04-22_10-53-28_sorted/curated/experiment1_Record Node 104#Neuropix-PXI-100.ProbeA_recording1'

In [ ]:
os.path.exists(session_dir['sorted_dir_raw'])

True

In [ ]:
session

'behavior_785956_2025-05-23_13-45-21'

In [ ]:
process('behavior_758018_2025-03-19_11-16-44', data_type='raw')

behavior_758018_2025-03-19_11-16-44 start
107 out of 201 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


KeyboardInterrupt: 

In [ ]:
# make a df from all_Tm, columns correspond to focus regressors
 